In [4]:
import os
import time
from datetime import datetime
import argparse
import gymnasium as gym
import numpy as np
import torch as th
import pandas as pd
import csv

from gym_pybullet_drones.utils.Logger import Logger
from gym_pybullet_drones.envs.HoverAviary import HoverAviary
from gym_pybullet_drones.envs.MultiHoverAviary import MultiHoverAviary
from gym_pybullet_drones.utils.utils import sync, str2bool
from gym_pybullet_drones.utils.enums import ObservationType, ActionType, Physics

from policies import GaussianMLPPolicy
from server import Federated_RL

DEFAULT_GUI = True
DEFAULT_RECORD_VIDEO = True
DEFAULT_OUTPUT_FOLDER = 'results'
DEFAULT_COLAB = False
DEFAULT_DYNAMICS = Physics('pyb_gnd_drag_dw') # pyb: Pybullet dynamics; dyn: Explicit Dynamics specified in BaseAviary.py
DEFAULT_WIND = np.array([0, 0.05, 0]) # units are in induced newtons
DEFAULT_OBS = ObservationType('kin') # 'kin' or 'rgb'
DEFAULT_ACT = ActionType('rpm') # 'rpm' or 'pid' or 'vel' or 'one_d_rpm' or 'one_d_pid'
DEFAULT_AGENTS = 2
DEFAULT_MA = False

DR = True
MASS_RANGE = [0.027, 0.042] # Maximum recommended payload is 15g
WIND_RANGE = 0.005 # Inspired by literature

In [5]:
algorithms = ['FedSVRPG-M', 'PPO', 'SAC', 'TD3']
num_agents = len(algorithms)
envs = [HoverAviary for _ in range(num_agents)]
env_kwargs = [dict(obs = DEFAULT_OBS, act = DEFAULT_ACT) for _ in range(num_agents)]
agent_names = algorithms
if DR == True:
    domain_randomizations = [DR for _ in range(num_agents)]
    DR_episode_thresholds = [.5 for _ in range(num_agents)] # Probability of DR at each episode
    DR_step_thresholds = [.3 for _ in range(num_agents)] # If DR episode, probability of wind at each step

mass_ranges = [MASS_RANGE for _ in range(num_agents)]
wind_ranges = [WIND_RANGE for _ in range(num_agents)]
env_example = HoverAviary(**env_kwargs[0])
# Get the state size
state_space = env_example.observation_space
state_size = state_space.shape[1]
# Get the action size
action_space = env_example.action_space
action_size = action_space.shape[1]

layers = [512, 512, 256, 128]
value_layers = [32, 32]
# Maintain consistent network structures
policy_kwargs = dict(activation_fn=th.nn.Tanh,
                     net_arch=dict(pi=layers, qf=value_layers))

print("State size:", state_size)
print("Action size:", action_size)

policy = GaussianMLPPolicy(input_size=state_size, output_size=action_size, hidden_layers=layers) # Will need some smarter way to initialize the policy within the model in the future
# ASSUMING ONE ALGORITHM SO FAR. WILL IMPLEMENT GENERAL STRUCTURE FOR DIVERSIFIED ALGORITHMS LATER

#### Train the model #######################################
model = Federated_RL(policy = policy,
                     envs = envs,
                     env_kwargs = env_kwargs,
                     num_agents = num_agents,
                     global_iterations = 20,
                     state_size = state_size,
                     action_size = action_size,
                     local_step_size = 1e-3,
                     policy_kwargs = policy_kwargs,
                     critic_net_aggregation = False,
                     critic_net = value_layers,
                     local_iterations = 10,
                     max_episode_length=2048,
                     agent_names = agent_names,
                     DR = domain_randomizations,
                     DR_episode_th = DR_episode_thresholds,
                     DR_step_th = DR_step_thresholds,
                     mass_ranges = mass_ranges,
                     wind_ranges = wind_ranges,
                     algorithms = algorithms)

State size: 72
Action size: 4


In [6]:
serverModel = model.learn()


Training agent FedSVRPG-M

Episode has activated Domain Randomization. Wind will be applied with a probability of 0.3 with a maximum magnitude of 0.005 Newtons.
Mass parameter for next episode: 0.035672848348311186
GLOBAL ITERATION: 0
LOCAL ITERATION: 0

Episode Reward: 43.6539754487209

Importance sampling weight: 0.0010000000474974513

GLOBAL ITERATION: 0
LOCAL ITERATION: 1

Episode Reward: 73.76353250134098

Importance sampling weight: 0.0010000000474974513

Episode has activated Domain Randomization. Wind will be applied with a probability of 0.3 with a maximum magnitude of 0.005 Newtons.
Mass parameter for next episode: 0.033335569654417904
GLOBAL ITERATION: 0
LOCAL ITERATION: 2

Episode Reward: 72.88044028650097

Importance sampling weight: 1.7999999523162842

Episode has activated Domain Randomization. Wind will be applied with a probability of 0.3 with a maximum magnitude of 0.005 Newtons.
Mass parameter for next episode: 0.028158931993396037
GLOBAL ITERATION: 0
LOCAL ITERATIO

/Users/kevinhan/opt/anaconda3/envs/drones/lib/python3.12/site-packages/stable_baselines3/common/callbacks.py:426: UserWarning: Training and eval env are not of the same type<stable_baselines3.common.vec_env.vec_check_nan.VecCheckNan object at 0x138b2a1b0> != <stable_baselines3.common.vec_env.dummy_vec_env.DummyVecEnv object at 0x137674aa0>
  warnings.warn("Training and eval env are not of the same type" f"{self.training_env} != {self.eval_env}")


Eval num_timesteps=2048, episode_reward=22.25 +/- 0.53
Episode length: 16.20 +/- 0.40
New best mean reward!
Episode has activated Domain Randomization. Wind will be applied with a probability of 0.3 with a maximum magnitude of 0.005 Newtons.
Mass parameter for next episode: 0.02956938218724693
Eval num_timesteps=4096, episode_reward=24.64 +/- 0.00
Episode length: 18.00 +/- 0.00
New best mean reward!
Episode has activated Domain Randomization. Wind will be applied with a probability of 0.3 with a maximum magnitude of 0.005 Newtons.
Mass parameter for next episode: 0.039798587683300836
Eval num_timesteps=6144, episode_reward=26.03 +/- 0.01
Episode length: 19.00 +/- 0.00
New best mean reward!
Eval num_timesteps=8192, episode_reward=31.55 +/- 0.00
Episode length: 23.00 +/- 0.00
New best mean reward!
Episode has activated Domain Randomization. Wind will be applied with a probability of 0.3 with a maximum magnitude of 0.005 Newtons.
Mass parameter for next episode: 0.038966393228548606
Eval 

/Users/kevinhan/opt/anaconda3/envs/drones/lib/python3.12/site-packages/stable_baselines3/common/callbacks.py:426: UserWarning: Training and eval env are not of the same type<stable_baselines3.common.vec_env.vec_check_nan.VecCheckNan object at 0x138b2a1b0> != <stable_baselines3.common.vec_env.dummy_vec_env.DummyVecEnv object at 0x13742ae40>
  warnings.warn("Training and eval env are not of the same type" f"{self.training_env} != {self.eval_env}")


Eval num_timesteps=2048, episode_reward=41.83 +/- 0.41
Episode length: 32.20 +/- 0.40
New best mean reward!
Eval num_timesteps=4096, episode_reward=37.27 +/- 0.00
Episode length: 28.00 +/- 0.00
Eval num_timesteps=6144, episode_reward=33.55 +/- 0.01
Episode length: 25.00 +/- 0.00
Eval num_timesteps=8192, episode_reward=31.08 +/- 0.00
Episode length: 23.00 +/- 0.00
Eval num_timesteps=10240, episode_reward=29.83 +/- 0.01
Episode length: 22.00 +/- 0.00
Eval num_timesteps=12288, episode_reward=29.91 +/- 0.00
Episode length: 22.00 +/- 0.00
Episode has activated Domain Randomization. Wind will be applied with a probability of 0.3 with a maximum magnitude of 0.005 Newtons.
Mass parameter for next episode: 0.03211406489473048
Eval num_timesteps=14336, episode_reward=28.69 +/- 0.00
Episode length: 21.00 +/- 0.00
Episode has activated Domain Randomization. Wind will be applied with a probability of 0.3 with a maximum magnitude of 0.005 Newtons.
Mass parameter for next episode: 0.03478060344604218

...

Eval num_timesteps=2048, episode_reward=361.61 +/- 20.27
Episode length: 214.60 +/- 8.87
New best mean reward!
Episode has activated Domain Randomization. Wind will be applied with a probability of 0.3 with a maximum magnitude of 0.005 Newtons.
Mass parameter for next episode: 0.03944756252667968
Eval num_timesteps=4096, episode_reward=355.29 +/- 22.78
Episode length: 214.40 +/- 10.87
Episode has activated Domain Randomization. Wind will be applied with a probability of 0.3 with a maximum magnitude of 0.005 Newtons.
Mass parameter for next episode: 0.03501887452037218
Eval num_timesteps=6144, episode_reward=336.49 +/- 0.67
Episode length: 210.20 +/- 0.40
Eval num_timesteps=8192, episode_reward=282.17 +/- 14.48
Episode length: 183.00 +/- 6.45
Episode has activated Domain Randomization. Wind will be applied with a probability of 0.3 with a maximum magnitude of 0.005 Newtons.
Mass parameter for next episode: 0.03277660430414321
Eval num_timesteps=10240, episode_reward=248.39 +/- 2.05
Epis

/Users/kevinhan/opt/anaconda3/envs/drones/lib/python3.12/site-packages/stable_baselines3/common/callbacks.py:426: UserWarning: Training and eval env are not of the same type<stable_baselines3.common.vec_env.vec_check_nan.VecCheckNan object at 0x138b2a1b0> != <stable_baselines3.common.vec_env.dummy_vec_env.DummyVecEnv object at 0x137675670>
  warnings.warn("Training and eval env are not of the same type" f"{self.training_env} != {self.eval_env}")


Eval num_timesteps=2048, episode_reward=353.26 +/- 16.52
Episode length: 210.60 +/- 8.04
New best mean reward!
Episode has activated Domain Randomization. Wind will be applied with a probability of 0.3 with a maximum magnitude of 0.005 Newtons.
Mass parameter for next episode: 0.03806569177045957
Eval num_timesteps=4096, episode_reward=329.15 +/- 11.43
Episode length: 200.40 +/- 4.54
Episode has activated Domain Randomization. Wind will be applied with a probability of 0.3 with a maximum magnitude of 0.005 Newtons.
Mass parameter for next episode: 0.0341194998935455
Eval num_timesteps=6144, episode_reward=315.23 +/- 6.53
Episode length: 193.80 +/- 1.94
Eval num_timesteps=8192, episode_reward=268.39 +/- 3.21
Episode length: 173.40 +/- 0.80
Eval num_timesteps=10240, episode_reward=245.45 +/- 15.38
Episode length: 161.40 +/- 7.14
Eval num_timesteps=12288, episode_reward=189.65 +/- 14.30
Episode length: 184.60 +/- 49.53
Episode has activated Domain Randomization. Wind will be applied with 

/Users/kevinhan/opt/anaconda3/envs/drones/lib/python3.12/site-packages/stable_baselines3/common/callbacks.py:426: UserWarning: Training and eval env are not of the same type<stable_baselines3.common.vec_env.vec_check_nan.VecCheckNan object at 0x138b2a1b0> != <stable_baselines3.common.vec_env.dummy_vec_env.DummyVecEnv object at 0x138b3e000>
  warnings.warn("Training and eval env are not of the same type" f"{self.training_env} != {self.eval_env}")


Eval num_timesteps=2048, episode_reward=361.83 +/- 18.24
Episode length: 214.80 +/- 7.63
New best mean reward!
Eval num_timesteps=4096, episode_reward=385.96 +/- 28.36
Episode length: 229.20 +/- 13.08
New best mean reward!
Episode has activated Domain Randomization. Wind will be applied with a probability of 0.3 with a maximum magnitude of 0.005 Newtons.
Mass parameter for next episode: 0.031217581621207222
Eval num_timesteps=6144, episode_reward=414.40 +/- 0.66
Episode length: 242.00 +/- 0.00
New best mean reward!
Eval num_timesteps=8192, episode_reward=411.85 +/- 0.67
Episode length: 242.00 +/- 0.00
Episode has activated Domain Randomization. Wind will be applied with a probability of 0.3 with a maximum magnitude of 0.005 Newtons.
Mass parameter for next episode: 0.030676564656532224
Eval num_timesteps=10240, episode_reward=404.08 +/- 2.90
Episode length: 242.00 +/- 0.00
Eval num_timesteps=12288, episode_reward=268.31 +/- 32.15
Episode length: 205.60 +/- 28.71
Episode has activated D

In [10]:
rewards = model.get_rewards()
np.savetxt("TotalRewards_NoCritic.csv", rewards, delimiter=',')
evaluated_rewards = np.array(rewards)
print(evaluated_rewards.shape)

(80, 10)


In [11]:
FEDSVRPG_M_rewards = []
PPO_rewards = []
SAC_rewards = []
TD3_rewards = []
reward_lists = [FEDSVRPG_M_rewards, PPO_rewards, SAC_rewards, TD3_rewards]
for i, rewards in enumerate(evaluated_rewards):
    reward_lists[i % 4].append(rewards)

In [12]:
np.savetxt('FEDSVRPG_M_rewards_without_value_aggregation.csv',FEDSVRPG_M_rewards, delimiter=',')
np.savetxt('PPO_rewards_without_value_aggregation.csv', PPO_rewards, delimiter=',')
np.savetxt('SAC_rewards_without_value_aggregation.csv', SAC_rewards, delimiter=',')
np.savetxt('TD3_rewards_without_value_aggregation.csv', TD3_rewards, delimiter=',')